# 실험 주제

resnet18을 통해 분류된 데이터에서, grad-cam을 추출한다.

grad-cam과 normalized mel-spectrogram을 더해서, 다시 resnet18을 train한다.

- http://keunwoochoi.blogspot.com/2016/03/2.html
- http://www.rex-ai.info/docs/AI_Example_CNN_speech_recognize
- https://www.youtube.com/watch?v=oltGIc4uo5c
- https://youdaeng-com.tistory.com/5
- https://quokkas.tistory.com/37 : early stopping
- https://continuous-development.tistory.com/166 : stratified kfold
- https://deep-learning-study.tistory.com/476 fiter 시각화
- https://wyatt37.tistory.com/10 : random over sampling

In [ ]:
!pip install openpyxl

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn # 인공 신경망 모델들 모아놓은 모듈
import torch.nn.functional as F #그중 자주 쓰이는것들을 F로
from torchvision import transforms, datasets
import cv2
from torchvision import transforms, datasets
import pandas as pd
import openpyxl
import os
from glob import glob
import torchvision.models as models
import sys

p = os.path.abspath('../..') # 상위 폴더를 사용하기 위해서.
sys.path.insert(1, p)
from pytorchtools.pytorchtools import EarlyStopping # 상위 폴더에 추가된 모듈.


if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
#DEVICE = torch.device('cpu')
print('Using Pytorch version : ',torch.__version__,' Device : ',DEVICE)

In [ ]:
import numpy as np
import librosa, librosa.display 
import matplotlib.pyplot as plt
#window sizde : FFT를 할때 참조할 그래프 길이 ( 프레임 하나당 sample 수 )
#자연어 처리에서는 25ms 사용. https://ahnjg.tistory.com/93
#초당 50000hz 중 1250개씩 윈도우 사이즈로 사용.
sr=50000
win_length =  np.int64(50000/40) # 1250
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기(sample)를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다.
#hop_length가 mfcc의 frame수를 결정한다.

# SVD 문장 데이터에서 Feature 추출
- mfcc

# 데이터 나누기 - Stratified KFold

- pathology : 1194 / healthy : 634 / 총 1828
- k = 5

## 1. test/ train 나누기


In [ ]:
#1. train, test 나누기

import os
from glob import glob
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.model_selection import train_test_split # train , test 분리에 사용.


pathology = glob('../../../voice_data/fusion/pathology/phrase/*.wav')
healthy = glob('../../../voice_data/fusion/healthy/phrase/*.wav')
print("Pathology : ",len(pathology))
print("Healthy: ",len(healthy))

X = pathology+healthy # path 데이터 합
print("총 데이터수 : ",len(X))
Y = [] # 라벨
for idx,x in enumerate(X):
    if idx<1193:
        Y.append("pathology")
    else:
        Y.append("healthy")

X, X_test, Y, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, stratify=Y, random_state=456)
#stratify를 넣어서, test에도 라벨별 잘 분류되게 한다.

print("---")
print("훈련 셋 : ",len(Y),Counter(Y))
print("테스트 셋 : ",len(Y_test),Counter(Y_test))
print("---")

# 2. train set 모두 mel-spectro , grad-cam 구해주기.

## 2. random over sampling

In [ ]:
#2. random over sampling

X = np.array(X).reshape(-1,1)#각 데이터를 다 행으로 넣음. (1194,1)
#Y = np.array(Y)
ros = RandomOverSampler(random_state = 123)
X_res,Y_res = ros.fit_resample(X,Y)

print('before dataset shape {}'.format(Counter(Y)) )
print('Resampled dataset shape {}'.format(Counter(Y_res)) )


In [ ]:
#원래대로 돌리기
X=X_res.reshape(1, -1)
print( '총 데이터수 : ',X[0].size )
print(  '복사된 수 : ',X[0].size - np.unique(X[0]).size )

X=X[0].tolist()
Y=Y_res

In [ ]:
print(X[0:4])
print(Y[0:4])

In [ ]:
X

## 3. stratified k-fold

In [ ]:
#1. train, test 나누기
#stratified kfold
import os
import random #데이터 shuffle 사용
from glob import glob
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler
from collections import Counter



skf_iris = StratifiedKFold(n_splits=5,shuffle=True,random_state=456)
cnt_iter = 0

X_train_list = [] #데이터 셋 보관
Y_train_list = []

X_valid_list = []
Y_valid_list = []

for train_idx, test_idx in skf_iris.split(X,Y):
    
    #split으로 반환된 인덱스를 이용하여, 학습 검증용 테스트 데이터 추출
    cnt_iter += 1
    X_train, X_valid = [X[idx] for idx in train_idx.tolist() ], [X[idx] for idx in test_idx.tolist() ]
    Y_train, Y_valid = [Y[idx] for idx in train_idx.tolist() ], [Y[idx] for idx in test_idx.tolist() ]
    
    X_train_list.append(X_train)
    X_valid_list.append(X_valid)
    
    Y_train_list.append(Y_train)
    Y_valid_list.append(Y_valid)
    
    
    #학습 및 예측
    
    label_train = Y_train
    label_test = Y_valid
    unique_train, train_counts = np.unique(label_train, return_counts = True)
    unique_test, test_counts = np.unique(label_test, return_counts = True)
    
    uniq_cnt_train = dict(zip(unique_train, train_counts))
    uniq_cnt_test = dict(zip(unique_test, test_counts))
    
    
    
    print('교차 검증 : {}'.format(cnt_iter))
    print('학습 레이블 데이터 분포 : \n', uniq_cnt_train)
    print('검증 레이블 데이터 분포 : \n', uniq_cnt_test,'\n')



# 데이터 정의
- 추가적으로 데이터의 크기를 맞춰주기 위해 3초로 padding 및 truncate 실시 https://sequencedata.tistory.com/25 FixAudioLength
- 논문에서는 400frame으로 설정.(여기서는 500frame)
- 전처리 방법 결정.


In [ ]:
from torch.utils.data import Dataset, DataLoader

classes = ["pathology","healthy"]
sr=50000
win_length =  np.int64(50000/40) # 1250
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기(sample)를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다.
#hop_length가 mfcc의 frame수를 결정한다.



class svd_dataset(Dataset):
    def __init__(self,data_path_list,classes,data_num,training,transform=None):
        #클래스에서 사용할 인자를 받아 인스턴스 변수로 저장하는 일을 한다.
        #예를들면, 이미지의 경로 리스트를 저장하는 일을 하게 된다.
        
        #data_num : k 개 데이터 셋 중 어떤것을 쓸지
        #test인지 아닌지.
        
        self.path_list = data_path_list[data_num]
        self.data_num = data_num
        self.training = training
        self.label = svd_dataset.get_label(self.path_list,training,data_num)
        self.classes=classes
        self.transform=transform
        
    
    @classmethod
    def get_label(cls,data_path_list,training,data_num):
        label_list=[]
        
        if training:
            for idx,x in enumerate(data_path_list):
                label_list.append(Y_train_list[data_num][idx])
        else:
            for idx,x in enumerate(data_path_list):
                label_list.append(Y_valid_list[data_num][idx])
        #print(label_list)
        return label_list
    
    
    def __len__(self):
        return len(self.path_list)
        #데이터 셋의 길이를 정수로 반환한다.     
    
       
    def __getitem__(self, idx):
        """
        1. path를 받아서, 소리에서 mfcc를 추출
        2. mfcc를 224프레임으로 패딩.
        3. resnet에 사용되기 위해 3채널로 복사(rgb 처럼)
        4. 0~1 정규화
        
        """
        sig, sr = librosa.load(self.path_list[idx], sr=50000)# 논문에서 f_s = 50 000HZ
        #stft 500 FRAME이 되도록 패딩.
        length = 300
        mel_feature = librosa.feature.melspectrogram(sig,sr=sr,hop_length=hop_length,n_fft=n_fft)
        mel_feature = librosa.core.power_to_db(mel_feature,ref=np.max)
        pad2d = lambda a, i: a[:, 0:i] if a.shape[1] > i else np.hstack((a, np.zeros((a.shape[0], i-a.shape[1]))))
        mel_feature = pad2d(mel_feature, length)
        
        
        if self.transform:
            #print('transform')
            mel_feature=self.transform(mel_feature).type(torch.float32)# 데이터 0~1 정규화
            mel_feature=torch.stack([mel_feature,mel_feature,mel_feature])# 3채널로 복사.
            mel_feature = mel_feature.squeeze(dim=1)
        else:
            #print("else")
            mel_feature = torch.from_numpy(mel_feature).type(torch.float32)
            mel_feature=mel_feature.unsqueeze(0)#cnn 사용위해서 추가
            #MFCCs = MFCCs.permute(2, 0, 1)
            
        return mel_feature, self.classes.index(self.label[idx]),self.path_list[idx]
    

In [ ]:
from torch.utils.data import Dataset, DataLoader

classes = ["pathology","healthy"]
sr=50000
win_length =  np.int64(50000/40) # 1250
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기(sample)를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다.
#hop_length가 mfcc의 frame수를 결정한다.

# test set 제작을 위한 class
class svd_test_set(Dataset):
    def __init__(self,data_path_list,classes,transform=None):
        #클래스에서 사용할 인자를 받아 인스턴스 변수로 저장하는 일을 한다.
        #예를들면, 이미지의 경로 리스트를 저장하는 일을 하게 된다.
        
        #data_num : k 개 데이터 셋 중 어떤것을 쓸지
        #test인지 아닌지.
        
        self.path_list = data_path_list
        self.label = svd_test_set.get_label(self.path_list)
        self.classes=classes
        self.transform=transform
        
    
    @classmethod
    def get_label(cls,data_path_list):
        label_list=[]
        
        for idx,x in enumerate(data_path_list):
            label_list.append(Y_test[idx])
        #print(label_list)
        return label_list
    
    def __len__(self):
        return len(self.path_list)
        #데이터 셋의 길이를 정수로 반환한다. 
        
    def __getitem__(self, idx):
        """
        1. path를 받아서, 소리에서 mfcc를 추출
        2. mfcc를 224프레임으로 패딩.
        3. resnet에 사용되기 위해 3채널로 복사(rgb 처럼)
        4. 0~1 정규화
        
        """
        sig, sr = librosa.load(self.path_list[idx], sr=50000)# 논문에서 f_s = 50 000HZ
        
        #stft 500 FRAME이 되도록 패딩.
        length = 300
        mel_feature = librosa.feature.melspectrogram(sig,sr=sr,hop_length=hop_length,n_fft=n_fft)
        mel_feature = librosa.core.power_to_db(mel_feature,ref=np.max)
        pad2d = lambda a, i: a[:, 0:i] if a.shape[1] > i else np.hstack((a, np.zeros((a.shape[0], i-a.shape[1]))))
        mel_feature = pad2d(mel_feature, length)
        
        
        if self.transform:
            #print('transform')
            mel_feature=self.transform(mel_feature).type(torch.float32)# 데이터 0~1 정규화
            mel_feature=torch.stack([mel_feature,mel_feature,mel_feature])# 3채널로 복사.
            mel_feature = mel_feature.squeeze(dim=1)
        else:
            #print("else")
            mel_feature = torch.from_numpy(mel_feature).type(torch.float32)
            mel_feature=mel_feature.unsqueeze(0)#cnn 사용위해서 추가
            #MFCCs = MFCCs.permute(2, 0, 1)
            
        #결과로 마지막에는 path도 출력하게 수정.
        return mel_feature, self.classes.index(self.label[idx]), self.path_list[idx]
    
    

# 데이터 로더

- grad-cam 사용시 train은 완료했으니, valid test만 가져온다.

In [ ]:
#3. 하이퍼 파라미터
BATCH_SIZE =  1 #한 배치당 1개. 왜냐하면 test시 한개만 볼것이라서.
EPOCHS = 40 # 전체 데이터 셋을 50번 반복

In [ ]:
validation_loader = torch.utils.data.DataLoader(dataset = 
                                           svd_dataset(
                                               X_valid_list,
                                               classes,
                                               transform = transforms.ToTensor(),
                                               data_num=1,
                                               training=False
                                           ),
                                           batch_size = BATCH_SIZE,
                                           shuffle = True,) 

In [ ]:
# 테스트 데이터 로더.

test_loader = torch.utils.data.DataLoader(dataset = 
                                               svd_test_set(
                                                   X_test,
                                                   classes,
                                                   transform = transforms.ToTensor(),
                                               ),
                                               batch_size = BATCH_SIZE,
                                               shuffle = True,) 

# 데이터 확인

grad-map은  valid, test set에서만 확인해본다.

In [ ]:
#valid set 확인
#1개 배치 전체 다 가져오는 것.
for (valid_data,valid_label,path) in validation_loader:
    print("X_valid : ",valid_data.size(),'type:',valid_data.type())
    print("Y_valid : ",valid_label.size(),'type:',valid_label.type())
    print("path : ",path)
    break


librosa.display.specshow(valid_data[0][0].numpy(), sr=sr, hop_length=hop_length)
plt.xlabel("Time")
plt.ylabel("Frequency")
plt.colorbar(format='%+2.0f dB')
plt.title("Spectrogram (dB)")

#batch: 32 / 3채널 / frame수: 500  /  feature수: 13

In [ ]:
#test set 확인
#1개 배치 전체 다 가져오는 것.
for (test_data,test_label,path) in test_loader:
    print("X_test : ",test_data.size(),'type:',test_data.type())
    print("Y_test : ",test_label.size(),'type:',test_label.type())
    print("path : ",path)
    break


librosa.display.specshow(test_data[0][0].numpy(), sr=sr, hop_length=hop_length)
plt.xlabel("Time")
plt.ylabel("Frequency")
plt.colorbar(format='%+2.0f dB')
plt.title("Spectrogram (dB)")

#batch: 32 / 3채널 / frame수: 500  /  feature수: 13

In [ ]:
#데이터 확인
pd.DataFrame(test_data[0][0].numpy())

# grad cam
- https://www.secmem.org/blog/2020/01/17/gradcam/
- https://blog.naver.com/PostView.nhn?blogId=domodal&logNo=221485292052&parentCategoryNo=&categoryNo=10&viewDate=&isShowPopularPosts=true&from=search
- f1-score가 가장높은 5번 모델 cam 확인
- 마지막 feature map 부터 output 까지의 gradient를 계산해야한다.
- https://sanghyu.tistory.com/46 normalize
- https://github.com/PeterKim1/paper_code_review/blob/master/9.%20Visual%20Explanations%20from%20Deep%20Networks%20via%20Gradient-based%20Localization(Grad-CAM)/Grad-CAM_pretrained.ipynb

In [ ]:
# final conv layer name 
finalconv_name = 'layer4'

# activations
feature_blobs = []

# gradient를 가져올 hook 함수
backward_feature = []

transform_norm = transforms.ToTensor()

# output으로 나오는 feature를 feature_blobs에 append하도록
def hook_feature(module, input, output):
    feature_blobs.append(output.cpu().data.numpy()) # 레이어의 마지막 output(피처맵)을 구하는 함수
    

# Grad-CAM
def backward_hook(module, input, output):
    backward_feature.append(output[0])  #backward시에 그래디언트 저장.

In [ ]:
# 모델 수정해서 hook 추가 사용.



def model_initialize():
    model = models.resnet18(pretrained=True).cuda()
    model.ftrs = model.fc.in_features # in_features : fully connected의 입력수.
    num_ftrs = model.fc.in_features
    
    model.layer4.register_forward_hook(hook_feature)
    model.layer4.register_backward_hook(backward_hook)
    
    model.fc = nn.Sequential(nn.Linear(num_ftrs, 256),
                             nn.BatchNorm1d(256),
                             nn.ReLU(),
                             nn.Dropout(p=0.5),
                             nn.Linear(256,128),
                             nn.BatchNorm1d(128),
                             nn.ReLU(),
                             nn.Dropout(p=0.5),
                             nn.Linear(128,64),
                             nn.BatchNorm1d(64),
                             nn.ReLU(),
                             nn.Dropout(p=0.5),
                             nn.Linear(64,50),
                             nn.BatchNorm1d(50),
                             nn.ReLU(),
                             nn.Dropout(p=0.5),
                             nn.Linear(50,2)
                            )

    model = model.cuda()
    return model
model=model_initialize()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)

check_path = '../../checkpoint/checkpoint_melspectro_resnet18_true_ros_2_300.pt'
model.load_state_dict(torch.load(check_path))
print(model)


마지막 cnn layer는 layer4로 볼 수 있다.

## validation set 확인

- validation의 cam을 반영하여, test set의 성능을 높히는 것이 목표.

In [ ]:
from torch.autograd import Variable
from torchvision.utils import make_grid, save_image



#pathology 음성 파일 가져오기

length=300
image_list=[]

label_list=[]
prob_list=[] #valid set의 확률값
name_list=[] #파일명
predict_label_list=[] #예측 라벨
res = []



mel_list=[]#정규 mel-spectrogram 모은 리스트
fallout_list=[] # cam - mel . 음수는 0 처리.
grad_list=[]#grad-cam을 담는 배열. amp sum 확인용


model.eval()
mel_sample=pathology[0]
for image, label, path in validation_loader:
    
    name = path[0].split('\\')[-1]
    name = name.replace('.wav', '')
    name_list.append(name)
    label_list.append(classes[label])
    
    # activations
    feature_blobs = []
    # gradient를 가져올 hook 함수
    backward_feature = []
    
    mel_sample = image
    mel_sample = mel_sample.to(DEVICE).float()


    model._modules.get(finalconv_name).register_forward_hook(hook_feature)
    model._modules.get(finalconv_name).register_backward_hook(backward_hook)



    # Prediction
    logit = model(mel_sample) # 예측값 구하기.
    out = F.softmax(logit, dim=1).data.squeeze() # softmax 적용 (모델을 통과는 했지만, criterion는 안통과함.)
    probs, idx = out.sort(0, True)
    print("Predicted label : %d, Probability : %.2f" % (idx[0].item(), probs[0].item()))
    predict_label_list.append(classes[idx[0].item()])
    prob_list.append(probs[0].item())
    res.append(classes[label]==classes[idx[0].item()])
    

    ###########
    # Grad - cam
    ###########

    score = logit[:, idx[0]].squeeze() # 예측값 y^c.
    score.backward(retain_graph = True) # 예측값 y^c에 대해서 backprop 진행
    
    activations = torch.Tensor(feature_blobs[0]).to(DEVICE) # (1, 512, 7, 7), forward activations. append라서 0번이 마지막
    gradients = backward_feature[0] # (1, 512, 4, 10), backward gradients. 마지막 conv layer의 gradient
    b, k, u, v = gradients.size()  # batch, 피처맵 수,  상, 하
    #print(gradients.size())

    alpha = gradients.view(b, k, -1).mean(2) # (1, 512, 7*7) => (1, 512), feature map k의 'importance'
    weights = alpha.view(b, k, 1, 1) # (1, 512, 1, 1)

    grad_cam_map = (weights*activations).sum(1, keepdim = True) # alpha * A^k = (1, 512, 7, 7) => (1, 1, 7, 7)
    grad_cam_map = F.relu(grad_cam_map) # Apply R e L U
    grad_cam_map = F.interpolate(grad_cam_map, size=(128, 300), mode='bilinear', align_corners=False) # (1, 1, 128, 300)
    map_min, map_max = grad_cam_map.min(), grad_cam_map.max()
    grad_cam_map = (grad_cam_map - map_min).div(map_max - map_min).data # (1, 1, 128, 300), min-max scaling

    grad_heatmap=grad_cam_map.squeeze().cpu()
    grad_list.append(grad_heatmap)
    
    # grad_cam_map.squeeze() : (128, 300)

    #save_image(grad_heatmap, os.path.join("./", "Grad_CAM.jpg"))
    
    # mel-spectorgram min-max normalization
    
    mel_min,mel_max = mel_sample.squeeze()[0].min(),mel_sample.squeeze()[0].max()
    mel_sample = (mel_sample.squeeze()[0]- mel_min).div(mel_max-mel_min)
    mel_sample = mel_sample.cpu().numpy()
    mel_list.append(mel_sample)
    
    
    #mel_sample = librosa.util.normalize(mel_sample.cpu().squeeze().numpy()[0]) #수정 필요. min-max normalization

    grad_result = grad_heatmap.numpy() + mel_sample # (1, 3, 244, 244)
    
    fallout_list.append(grad_heatmap.numpy() - mel_sample)


    plt.figure(figsize=(10,5))

    plt.subplot(131)
    librosa.display.specshow(mel_sample, sr=sr, hop_length=hop_length)
    plt.xlabel("Time")
    plt.ylabel("Frequency")
    plt.colorbar(format='%+2.0f dB')
    plt.title("Spectrogram (dB)")

    plt.subplot(132)
    librosa.display.specshow(grad_heatmap.numpy(), sr=sr, hop_length=hop_length)
    plt.xlabel("Time")
    plt.ylabel("Frequency")
    plt.colorbar(format='%+2.0f dB')
    plt.title("Spectrogram (dB)")

    plt.subplot(133)
    librosa.display.specshow(grad_result, sr=sr, hop_length=hop_length)
    plt.xlabel("Time")
    plt.ylabel("Frequency")
    plt.colorbar(format='%+2.0f dB')
    plt.title("Spectrogram (dB)")
    
    
    
    
    if probs[0].item()<0.8:
        folder_path='./valid_images/cam_images_0.8/'
    elif probs[0].item()<0.9:
        folder_path='./valid_images/cam_images_0.9/'
    else:
        folder_path='./valid_images/cam_images_1.0/'
    
    plt.savefig(folder_path+name)
    plt.close()

result_excel=pd.concat( [pd.DataFrame(name_list),
            pd.DataFrame(label_list),
            pd.DataFrame(predict_label_list),
            pd.DataFrame(prob_list),
            pd.DataFrame(res)],axis=1)
result_excel.columns=['name','class','predict','probability','result']
result_excel.to_excel("./result_excel_valid.xlsx",index=False)



In [ ]:
#낙폭 feature의 max값 계산.
#1d array 계산 먼저.
#1d array를 구했으면, 최대값 또는 평균값을 가져오면 끝.

fallout_1d = []
for idx,fallout in enumerate(fallout_list):
    
    if prob_list[idx]<0.8:
        fallout_folder_path='./valid_images/fallout/cam_images_0.8/'
    elif prob_list[idx]<0.9:
        fallout_folder_path='./valid_images/fallout/cam_images_0.9/'
    else:
        fallout_folder_path='./valid_images/fallout/cam_images_1.0/'
    
    fallout_temp=fallout.max(axis=0) #projection
    fallout_temp=np.where(fallout_temp < 0 , 0, fallout_temp)
    fallout_1d.append(fallout_temp)
    
    plt.figure(figsize=(5,5))
    librosa.display.specshow(fallout, sr=sr, hop_length=hop_length)
    plt.xlabel("Time")
    plt.ylabel("Frequency")
    plt.colorbar(format='%+2.0f dB')
    plt.title("fallout")
    
    plt.savefig(fallout_folder_path+name_list[idx])
    plt.close()
    

In [ ]:
# amplitude sum과 probabilty의 상관성.
grad_ampsum = []
grad_area = []

for idx,gradcam in enumerate(grad_list):
    
    ampsum_temp=gradcam.sum().numpy() #amp_sum을 구하는방법
    gradcam=np.where(gradcam.numpy()>0,1,0)
    gradarea_temp = gradcam.sum()
    
    
    #print(fallout_temp)
    grad_ampsum.append(ampsum_temp)
    grad_area.append(gradarea_temp)



In [ ]:
# grad의 frequency 방향 projection
grad_frequency = []

for idx,gradcam in enumerate(grad_list):
    
    gradcam_temp=gradcam.numpy().max(axis=1) #projection
    gradcam_temp=np.mean(gradcam_temp)
    grad_frequency.append(gradcam_temp)
    
grad_frequency    


In [ ]:
# fallout frequency 방향 projection과 probabilty의 상관성.
#1d array 계산 먼저.
#1d array를 구했으면, 최대값 또는 평균값을 가져오면 끝.

fallout_frequency = []
for idx,fallout in enumerate(fallout_list):
    
    fallout_temp=fallout.max(axis=1) #projection
    fallout_temp=np.where(fallout_temp < 0 , 0, fallout_temp)
    fallout_temp=np.mean(fallout_temp)
    fallout_frequency.append(fallout_temp)

    
fallout_frequency

코드 수정해서 확률, 이름 데이터도 같이 저장.

그리고 일단 최대값 프로젝션으로 계속해보기.

In [ ]:
#fallout1d 저장.
import pickle



with open('./valid_images/fallout1d/fallout.pickle', 'wb') as f:
    pickle.dump(fallout_1d, f, pickle.HIGHEST_PROTOCOL)

with open('./valid_images/fallout1d/name.pickle', 'wb') as f:
    pickle.dump(name_list, f, pickle.HIGHEST_PROTOCOL)

with open('./valid_images/fallout1d/prob.pickle', 'wb') as f:
    pickle.dump(prob_list, f, pickle.HIGHEST_PROTOCOL)
    
with open('./valid_images/fallout1d/label.pickle', 'wb') as f:
    pickle.dump(label_list, f, pickle.HIGHEST_PROTOCOL)

with open('./valid_images/fallout1d/res.pickle', 'wb') as f:
    pickle.dump(res, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
with open('./valid_images/fallout1d/fallout.pickle', 'rb') as f:
    fallout_1d = pickle.load(f)

with open('./valid_images/fallout1d/name.pickle', 'rb') as f:
    name_list= pickle.load(f)

with open('./valid_images/fallout1d/prob.pickle', 'rb') as f:
    prob_list = pickle.load(f)

with open('./valid_images/fallout1d/label.pickle', 'rb') as f:
    label_list = pickle.load(f)


with open('./valid_images/fallout1d/res.pickle', 'rb') as f:
    res = pickle.load(f)

In [ ]:

fallout_mean = []    

for idx,fallout in enumerate(fallout_1d):
    print(name_list[idx],np.mean(fallout),prob_list[idx])
    fallout_mean.append(np.mean(fallout))
    
    if prob_list[idx]<0.8:
        fallout_folder_path='./valid_images/fallout1d/cam_images_0.8/'
    elif prob_list[idx]<0.9:
        fallout_folder_path='./valid_images/fallout1d/cam_images_0.9/'
    else:
        fallout_folder_path='./valid_images/fallout1d/cam_images_1.0/'
    
    plt.figure(figsize=(5,5))
    plt.plot(fallout)
    plt.xlabel("Time")
    plt.ylabel("Frequency")
    plt.title("fallout")
    
    plt.savefig(fallout_folder_path+name_list[idx])
    plt.close()


fallout_excel
요약 통계량 구하고,
산점도 나타내기.

In [ ]:
fallout_excel=pd.concat( [pd.DataFrame(name_list),
            pd.DataFrame(label_list),
            pd.DataFrame(prob_list),
            pd.DataFrame(fallout_mean),
            pd.DataFrame(res)],axis=1)
fallout_excel.columns=['name','class','probability','fallout','res']
fallout_excel.to_excel("./fallout_excel_valid.xlsx",index=False)

In [ ]:
fallout_excel=pd.read_excel("./fallout_excel_valid.xlsx")
fallout_excel

In [ ]:
#데이터 형태 변환
# 모두 pathology일 확률로 변환


# 1. healthy 맞춘 경우. 
filter1=fallout_excel['res']==True
filter2=fallout_excel['class']=='healthy'
fallout_excel['probability']=np.where(filter1&filter2,fallout_excel['probability'],1-fallout_excel['probability'])


#2. pathology 틀린 경우
filter1=fallout_excel['res']==False
filter2=fallout_excel['class']=='pathology'
fallout_excel['probability']=np.where(filter1&filter2,fallout_excel['probability'],1-fallout_excel['probability'])


#확인
filter1=fallout_excel['res']==True
filter2=fallout_excel['class']=='healthy'
fallout_excel[filter1&filter2]['probability']


In [ ]:
fallout_excel.plot(x='class',y='fallout', kind = 'scatter')

In [ ]:
fallout_excel['res']=fallout_excel['res'].astype(int)

In [ ]:
fallout_excel['fallout_frequency'] = pd.DataFrame(fallout_frequency)

In [ ]:
fallout_excel['grad_frequency'] = pd.DataFrame(grad_frequency)

In [ ]:
#amplitude sum, area 과 probability 상관성 확인을 위한 dataframe
ampsum_prob=pd.concat([ pd.DataFrame(grad_ampsum),
                       pd.DataFrame(grad_area),
                       fallout_excel['probability'],
                       fallout_excel['class'],
                       fallout_excel['fallout'],
                       pd.DataFrame(fallout_frequency)
                       ],axis=1)
ampsum_prob.columns=['amp_sum','grad_area','probability','class','fallout','fallout_frequency']
ampsum_prob



In [ ]:
#amplitude sum과 probability 상관성 확인
ampsum_prob[['amp_sum','probability']].corr()

In [ ]:
#grad_area과 probability 상관성 확인
ampsum_prob[['grad_area','probability']].corr()

In [ ]:
#grad_area과 fallout 상관성 확인
ampsum_prob[['grad_area','fallout']].corr()

In [ ]:
colors = {'pathology':'red', 'healthy':'green'}
ampsum_prob.plot(x='probability',y='amp_sum', kind = 'scatter',c=ampsum_prob['class'].map(colors))



In [ ]:
#amplitude sum을 변형한 넓이와 probs의 관계
colors = {'pathology':'red', 'healthy':'green'}
ampsum_prob.plot(x='probability',y='grad_area', kind = 'scatter',c=ampsum_prob['class'].map(colors))

In [ ]:
#amplitude sum을 변형한 넓이와 fallout 관계
colors = {'pathology':'red', 'healthy':'green'}
ampsum_prob.plot(x='fallout',y='grad_area', kind = 'scatter',c=ampsum_prob['class'].map(colors))

In [ ]:
#amplitude sum을 변형한 넓이와 fallout 관계
colors = {'pathology':'red', 'healthy':'green'}
ampsum_prob.plot(x='fallout',y='fallout_frequency', kind = 'scatter',c=ampsum_prob['class'].map(colors))

In [ ]:
filter1=fallout_excel['res']==0
filter2=fallout_excel['class']=='healthy'
fallout_filter1=fallout_excel[filter1&filter2]
fallout_filter1
#현재 validation set에서는 8건 밖에 healthy가 틀린경우가 없다.

In [ ]:
#전체 데이터.
#틀렸을때 어디 위치하는지.
#기준은 pathology의 probability
colors = {0:'red', 1:'green'}
fallout_excel.plot(x='probability',y='fallout', kind = 'scatter',c=fallout_excel['res'].map(colors))

In [ ]:
#fallout_freqency
colors = {0:'red', 1:'green'}
fallout_excel.plot(x='probability',y='fallout_frequency', kind = 'scatter',c=fallout_excel['res'].map(colors))

In [ ]:
#gradcam_freqency
colors = {0:'red', 1:'green'}
fallout_excel.plot(x='probability',y='grad_frequency', kind = 'scatter',c=fallout_excel['res'].map(colors))

In [ ]:
fallout_excel.plot(x='res',y='fallout', kind = 'scatter')

In [ ]:
fallout_excel.plot(x='res',y='fallout_frequency', kind = 'scatter')

In [ ]:
fallout_excel.corr()

In [ ]:
filter1=fallout_excel['probability']>0.9
filter2=fallout_excel['class']=='pathology'
fallout_filter1=fallout_excel[filter1&filter2]
fallout_filter1

In [ ]:
filter1=fallout_excel['probability']<0.5
filter2=fallout_excel['class']=='pathology'
filter3=fallout_excel['probability']>0.3
fallout_filter2=fallout_excel[filter1&filter2&filter3]
fallout_filter2

In [ ]:
#pathology인 데이터에서, pathology일 확률로 예측 한경우. 
#probability가 0.5 이하면 그냥 예측에 실패했다.
df1=pd.concat([fallout_filter1,fallout_filter2])
df1.plot(x='probability',y='fallout', kind = 'scatter',c=df1['res'].map(colors))

In [ ]:
#healthy가 맞는데, pathology라고 0.1 정도 된다고 생각한 경우 
filter1=fallout_excel['probability']<0.1
filter2=fallout_excel['class']=='healthy'
fallout_filter1=fallout_excel[filter1&filter2]
fallout_filter1

In [ ]:
#healthy가 맞는데, pathology라고 0.5~0.7라고 생각한 경우 
filter1=fallout_excel['probability']>0.5
filter2=fallout_excel['class']=='healthy'
#filter3=fallout_excel['probability']<0.8
fallout_filter2=fallout_excel[filter1&filter2]
fallout_filter2

In [ ]:
#healthy인데 pathology일 확률로 나타낸 그림
#예측이 틀린경우는 5건 밖에 없다.
#probability가 0.5 이하면 그냥 예측에 실패했다.
#healthy인데 pathology가 probabilty가 0.6 이상이라 하면, 그냥 틀려버린다.
df2=pd.concat([fallout_filter1,fallout_filter2])
df2.plot(x='probability',y='fallout', kind = 'scatter',c=df2['res'].map(colors))


In [ ]:
for idx,fallout in enumerate(fallout_1d):
    if prob_list[idx]>0.9:
        print(name_list[idx],np.mean(fallout),prob_list[idx])

In [ ]:
for idx,fallout in enumerate(fallout_1d):
    if prob_list[idx]<0.8:
        print(name_list[idx],np.mean(fallout),prob_list[idx])

In [ ]:
for idx,fallout in enumerate(fallout_1d):
    print(name_list[idx],np.max(fallout),prob_list[idx])

In [ ]:
with open('./valid_images/fallout1d/fallout.pickle', 'rb') as f:
    fallout_1d = pickle.load(f)
fallout_1d

In [ ]:
print(name_list[0])
plt.plot(fallout_1d[0])
print(np.mean(fallout_1d[0]))

## test set 확인

In [ ]:
from torch.autograd import Variable
from torchvision.utils import make_grid, save_image



#pathology 음성 파일 가져오기

length=300
image_list=[]

label_list=[]
prob_list=[] #valid set의 확률값
name_list=[] #파일명
predict_label_list=[] #예측 라벨
res = []



mel_list=[]#정규 mel-spectrogram 모은 리스트
fallout_list=[] # cam - mel . 음수는 0 처리.


model.eval()
mel_sample=pathology[0]
for image, label, path in test_loader:
    
    name = path[0].split('\\')[-1]
    name = name.replace('.wav', '')
    name_list.append(name)
    label_list.append(classes[label])
    
    # activations
    feature_blobs = []
    # gradient를 가져올 hook 함수
    backward_feature = []
    
    mel_sample = image
    mel_sample = mel_sample.to(DEVICE).float()


    model._modules.get(finalconv_name).register_forward_hook(hook_feature)
    model._modules.get(finalconv_name).register_backward_hook(backward_hook)

    

    # Prediction
    logit = model(mel_sample) # 예측값 구하기.
    out = F.softmax(logit, dim=1).data.squeeze() # softmax 적용 (모델을 통과는 했지만, criterion는 안통과함.)
    probs, idx = out.sort(0, True)
    print("Predicted label : %d, Probability : %.2f" % (idx[0].item(), probs[0].item()))
    predict_label_list.append(classes[idx[0].item()])
    prob_list.append(probs[0].item())
    res.append(classes[label]==classes[idx[0].item()])
    

    ###########
    # Grad - cam
    ###########

    score = logit[:, idx[0]].squeeze() # 예측값 y^c.
    score.backward(retain_graph = True) # 예측값 y^c에 대해서 backprop 진행
    
    activations = torch.Tensor(feature_blobs[0]).to(DEVICE) # (1, 512, 7, 7), forward activations. append라서 0번이 마지막
    gradients = backward_feature[0] # (1, 512, 4, 10), backward gradients. 마지막 conv layer의 gradient
    b, k, u, v = gradients.size()  # batch, 피처맵 수,  상, 하
    #print(gradients.size())

    alpha = gradients.view(b, k, -1).mean(2) # (1, 512, 7*7) => (1, 512), feature map k의 'importance'
    weights = alpha.view(b, k, 1, 1) # (1, 512, 1, 1)

    grad_cam_map = (weights*activations).sum(1, keepdim = True) # alpha * A^k = (1, 512, 7, 7) => (1, 1, 7, 7)
    grad_cam_map = F.relu(grad_cam_map) # Apply R e L U
    grad_cam_map = F.interpolate(grad_cam_map, size=(128, 300), mode='bilinear', align_corners=False) # (1, 1, 128, 300)
    map_min, map_max = grad_cam_map.min(), grad_cam_map.max()
    grad_cam_map = (grad_cam_map - map_min).div(map_max - map_min).data # (1, 1, 128, 300), min-max scaling

    grad_heatmap=grad_cam_map.squeeze().cpu()
    
    # grad_cam_map.squeeze() : (128, 300)

    #save_image(grad_heatmap, os.path.join("./", "Grad_CAM.jpg"))
    
    # mel-spectorgram min-max normalization
    
    mel_min,mel_max = mel_sample.squeeze()[0].min(),mel_sample.squeeze()[0].max()
    mel_sample = (mel_sample.squeeze()[0]- mel_min).div(mel_max-mel_min)
    mel_sample = mel_sample.cpu().numpy()
    mel_list.append(mel_sample)
    
    
    #mel_sample = librosa.util.normalize(mel_sample.cpu().squeeze().numpy()[0]) #수정 필요. min-max normalization

    grad_result = grad_heatmap.numpy() + mel_sample # (1, 3, 244, 244)
    
    fallout_list.append(grad_heatmap.numpy() - mel_sample)


    plt.figure(figsize=(10,5))

    plt.subplot(131)
    librosa.display.specshow(mel_sample, sr=sr, hop_length=hop_length)
    plt.xlabel("Time")
    plt.ylabel("Frequency")
    plt.colorbar(format='%+2.0f dB')
    plt.title("Spectrogram (dB)")

    plt.subplot(132)
    librosa.display.specshow(grad_heatmap.numpy(), sr=sr, hop_length=hop_length)
    plt.xlabel("Time")
    plt.ylabel("Frequency")
    plt.colorbar(format='%+2.0f dB')
    plt.title("Spectrogram (dB)")

    plt.subplot(133)
    librosa.display.specshow(grad_result, sr=sr, hop_length=hop_length)
    plt.xlabel("Time")
    plt.ylabel("Frequency")
    plt.colorbar(format='%+2.0f dB')
    plt.title("Spectrogram (dB)")
    
    
    
    
    if probs[0].item()<0.8:
        folder_path='./cam_images_0.8/'
    elif probs[0].item()<0.9:
        folder_path='./cam_images_0.9/'
    else:
        folder_path='./cam_images_1.0/'
    
    plt.savefig(folder_path+name)
    

result_excel=pd.concat( [pd.DataFrame(name_list),
            pd.DataFrame(label_list),
            pd.DataFrame(predict_label_list),
            pd.DataFrame(prob_list),
            pd.DataFrame(res)],axis=1)
result_excel.columns=['name','class','predict','probability','result']
result_excel.to_excel("./result_excel_test.xlsx",index=False)



In [ ]:
#낙폭 feature의 max값 계산.
#1d array 계산 먼저.
#1d array를 구했으면, 최대값 또는 평균값을 가져오면 끝.

fallout_1d = []
for idx,fallout in enumerate(fallout_list):
    
    if prob_list[idx]<0.8:
        fallout_folder_path='./fallout/cam_images_0.8/'
    elif prob_list[idx]<0.9:
        fallout_folder_path='./fallout/cam_images_0.9/'
    else:
        fallout_folder_path='./fallout/cam_images_1.0/'
    
    fallout_temp=fallout.max(axis=0) #projection
    fallout_temp=np.where(fallout_temp < 0 , 0, fallout_temp)
    fallout_1d.append(fallout_temp)
    
    plt.figure(figsize=(5,5))
    librosa.display.specshow(fallout, sr=sr, hop_length=hop_length)
    plt.xlabel("Time")
    plt.ylabel("Frequency")
    plt.colorbar(format='%+2.0f dB')
    plt.title("fallout")
    
    plt.savefig(fallout_folder_path+name_list[idx])
    

In [ ]:
#fallout1d 저장.
import pickle



with open('./fallout1d/fallout.pickle', 'wb') as f:
    pickle.dump(fallout_1d, f, pickle.HIGHEST_PROTOCOL)

with open('./fallout1d/name.pickle', 'wb') as f:
    pickle.dump(name_list, f, pickle.HIGHEST_PROTOCOL)

with open('./fallout1d/prob.pickle', 'wb') as f:
    pickle.dump(prob_list, f, pickle.HIGHEST_PROTOCOL)
    
with open('./fallout1d/label.pickle', 'wb') as f:
    pickle.dump(label_list, f, pickle.HIGHEST_PROTOCOL)

with open('./fallout1d/res.pickle', 'wb') as f:
    pickle.dump(res, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('./fallout1d/fallout.pickle', 'rb') as f:
    fallout_1d = pickle.load(f)

with open('./fallout1d/name.pickle', 'rb') as f:
    name_list= pickle.load(f)

with open('./fallout1d/prob.pickle', 'rb') as f:
    prob_list = pickle.load(f)

with open('./fallout1d/label.pickle', 'rb') as f:
    label_list = pickle.load(f)


with open('./fallout1d/res.pickle', 'rb') as f:
    res = pickle.load(f)

fallout_mean = []    

for idx,fallout in enumerate(fallout_1d):
    print(name_list[idx],np.mean(fallout),prob_list[idx])
    fallout_mean.append(np.mean(fallout))
    
    if prob_list[idx]<0.8:
        fallout_folder_path='./fallout1d/cam_images_0.8/'
    elif prob_list[idx]<0.9:
        fallout_folder_path='./fallout1d/cam_images_0.9/'
    else:
        fallout_folder_path='./fallout1d/cam_images_1.0/'
    
    plt.figure(figsize=(5,5))
    plt.plot(fallout)
    plt.xlabel("Time")
    plt.ylabel("Frequency")
    plt.title("fallout")
    
    plt.savefig(fallout_folder_path+name_list[idx])    


In [ ]:
fallout_excel=pd.concat( [pd.DataFrame(name_list),
            pd.DataFrame(label_list),
            pd.DataFrame(prob_list),
            pd.DataFrame(fallout_mean),
            pd.DataFrame(res)],axis=1)
fallout_excel.columns=['name','class','probability','fallout','res']
fallout_excel.to_excel("./fallout_excel_test.xlsx",index=False)

In [ ]:
fallout_excel=pd.read_excel("./fallout_excel_test.xlsx")
fallout_excel

In [ ]:
#데이터 형태 변환
# 모두 pathology일 확률로 변환


# 1. healthy 맞춘 경우. 
filter1=fallout_excel['res']==True
filter2=fallout_excel['class']=='healthy'
fallout_excel['probability']=np.where(filter1&filter2,fallout_excel['probability'],1-fallout_excel['probability'])


#2. pathology 틀린 경우
filter1=fallout_excel['res']==False
filter2=fallout_excel['class']=='pathology'
fallout_excel['probability']=np.where(filter1&filter2,fallout_excel['probability'],1-fallout_excel['probability'])


#확인
filter1=fallout_excel['res']==True
filter2=fallout_excel['class']=='healthy'
fallout_excel[filter1&filter2]['probability']


In [ ]:
fallout_excel.plot(x='class',y='fallout', kind = 'scatter')

In [ ]:
fallout_excel['res']=fallout_excel['res'].astype(int)

In [ ]:
filter1=fallout_excel['res']==0
filter2=fallout_excel['class']=='healthy'
fallout_filter1=fallout_excel[filter1&filter2]
print(fallout_filter1.shape[0])
fallout_filter1
#현재 test set에서는 42건 healthy가 틀린경우가 있다.

In [ ]:
filter1=fallout_excel['res']==0
filter2=fallout_excel['class']=='pathology'
fallout_filter1=fallout_excel[filter1&filter2]
print(fallout_filter1.shape[0])
fallout_filter1
#현재 test set에서는 23건 pathology가 틀린경우가 있다.

In [ ]:
#전체 데이터.
#틀렸을때 어디 위치하는지.
#기준은 pathology의 probability
colors = {0:'red', 1:'green'}
fallout_excel.plot(x='probability',y='fallout', kind = 'scatter',c=fallout_excel['res'].map(colors))

In [ ]:
fallout_excel.plot(x='res',y='fallout', kind = 'scatter')

In [ ]:
fallout_excel.corr()

In [ ]:
filter1=fallout_excel['probability']>0.9
filter2=fallout_excel['class']=='pathology'
fallout_filter1=fallout_excel[filter1&filter2]
fallout_filter1

In [ ]:
filter1=fallout_excel['probability']<0.5
filter2=fallout_excel['class']=='pathology'
filter3=fallout_excel['probability']>0.2
fallout_filter2=fallout_excel[filter1&filter2&filter3]
fallout_filter2

In [ ]:
#pathology인 데이터에서, pathology일 확률로 예측 한경우. 
#probability가 0.5 이하면 그냥 예측에 실패했다.
df1=pd.concat([fallout_filter1,fallout_filter2])
df1.plot(x='probability',y='fallout', kind = 'scatter',c=df1['res'].map(colors))

In [ ]:
#healthy가 맞는데, pathology라고 0.1 정도 된다고 생각한 경우 
filter1=fallout_excel['probability']<0.1
filter2=fallout_excel['class']=='healthy'
fallout_filter1=fallout_excel[filter1&filter2]
fallout_filter1

In [ ]:
#healthy가 맞는데, pathology라고 0.5~0.7라고 생각한 경우 
filter1=fallout_excel['probability']>0.5
filter2=fallout_excel['class']=='healthy'
#filter3=fallout_excel['probability']<0.8
fallout_filter2=fallout_excel[filter1&filter2]
fallout_filter2

In [ ]:
#healthy인데 pathology일 확률로 나타낸 그림
#예측이 틀린경우는 5건 밖에 없다.
#probability가 0.5 이하면 그냥 예측에 실패했다.
#healthy인데 pathology가 probabilty가 0.6 이상이라 하면, 그냥 틀려버린다.
df2=pd.concat([fallout_filter1,fallout_filter2])
df2.plot(x='probability',y='fallout', kind = 'scatter',c=df2['res'].map(colors))


#healthy는 fallout에 의존하지 않음을 알 수 있다.

In [ ]:
df2

# 데이터 추출

fallout < 0.4 이고, healthy로 예측한 데이터들을 엑셀로 추출한다.

그 데이터를 fusion network에 다시 삽입한다.

In [ ]:
#1. fallout < 0.4 이고, healthy로 예측한 데이터

filter1=fallout_excel['probability']<0.5
filter2=fallout_excel['fallout']<0.4

fallout_filter1=fallout_excel[filter1&filter2]
print(fallout_filter1['res'].shape[0],'건 추출.')
print(np.sum(fallout_filter1['res']),'건 맞춤.')
print(np.sum(fallout_filter1['res']==0),'건 틀림.')


fallout_filter1

#총 92건의 데이터를 다시 추론 할것이다.




In [ ]:
# 파일 이름 추출.
path_phrase = "../../../voice_data/fusion/"
fallout_fusion_data=fallout_filter1[['name','class']].copy()

fallout_fusion_data['name']=path_phrase+fallout_fusion_data['class']+'/phrase\\'+fallout_fusion_data['name']+'.wav'

In [ ]:
X_test = fallout_fusion_data['name'].to_list()
Y_test = fallout_fusion_data['class'].to_list()

In [ ]:
X_test

# 데이터 퓨전 네트워크 삽입

## 1. p a i u fusion

In [ ]:
from torch.utils.data import Dataset, DataLoader

#3. 하이퍼 파라미터
BATCH_SIZE =  16 #한 배치당 16개 음성데이터


classes = ["pathology","healthy"]
sr=50000
win_length =  np.int64(50000/40) # 1250
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기(sample)를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다.
#hop_length가 mfcc의 frame수를 결정한다.


# test set 제작을 위한 class
class svd_test_set(Dataset):
    def __init__(self,data_path_list,classes,transform=None):
        #클래스에서 사용할 인자를 받아 인스턴스 변수로 저장하는 일을 한다.
        #예를들면, 이미지의 경로 리스트를 저장하는 일을 하게 된다.
        
        #data_num : k 개 데이터 셋 중 어떤것을 쓸지
        #test인지 아닌지.
        
        self.path_list = data_path_list
        self.label = svd_test_set.get_label(self.path_list)
        self.classes=classes
        self.transform=transform
        
    
    @classmethod
    def get_label(cls,data_path_list):
        label_list=[]
        
        for idx,x in enumerate(data_path_list):
            label_list.append(Y_test[idx])
        #print(label_list)
        return label_list
    
    def __len__(self):
        return len(self.path_list)
        #데이터 셋의 길이를 정수로 반환한다. 
        
    def __getitem__(self, idx):
        """
        1. path를 받아서, 소리에서 mfcc를 추출
        2. mfcc를 224프레임으로 패딩.
        3. resnet에 사용되기 위해 3채널로 복사(rgb 처럼)
        4. 0~1 정규화
        
        """
        kind=self.path_list[idx].split('/')[-2]
        number=self.path_list[idx].split('\\')[-1].split('-')[0]
        path_a="../../../voice_data/fusion/"+kind+"/a\\"+number+"-a_n.wav"
        path_i="../../../voice_data/fusion/"+kind+"/i\\"+number+"-i_n.wav"
        path_u="../../../voice_data/fusion/"+kind+"/u\\"+number+"-u_n.wav"

        sig_phrase, sr = librosa.load(self.path_list[idx], sr=50000)# 논문에서 f_s = 50 000HZ
        sig_a, sr = librosa.load(path_a, sr=50000)# 논문에서 f_s = 50 000HZ
        sig_i, sr = librosa.load(path_i, sr=50000)# 논문에서 f_s = 50 000HZ
        sig_u, sr = librosa.load(path_u, sr=50000)# 논문에서 f_s = 50 000HZ
        
        
        
        
        #stft 500 FRAME이 되도록 패딩.
        length = 300
        mel_feature = librosa.feature.melspectrogram(sig_phrase,sr=sr,hop_length=hop_length,n_fft=n_fft)
        mel_feature = librosa.core.power_to_db(mel_feature,ref=np.max)
        
        mel_feature_a = librosa.feature.melspectrogram(sig_a,sr=sr,hop_length=hop_length,n_fft=n_fft)
        mel_feature_a = librosa.core.power_to_db(mel_feature_a,ref=np.max)
        
        mel_feature_i = librosa.feature.melspectrogram(sig_i,sr=sr,hop_length=hop_length,n_fft=n_fft)
        mel_feature_i = librosa.core.power_to_db(mel_feature_i,ref=np.max)
        
        mel_feature_u = librosa.feature.melspectrogram(sig_u,sr=sr,hop_length=hop_length,n_fft=n_fft)
        mel_feature_u = librosa.core.power_to_db(mel_feature_u,ref=np.max)
        
        
        
        #mel_feature=librosa.util.normalize(mel_feature) # l-infinity norm
        
        pad2d = lambda a, i: a[:, 0:i] if a.shape[1] > i else np.hstack((a, np.zeros((a.shape[0], i-a.shape[1]))))
        mel_feature = pad2d(mel_feature, length)
        mel_feature_a = pad2d(mel_feature_a, length)
        mel_feature_i = pad2d(mel_feature_i, length)
        mel_feature_u = pad2d(mel_feature_u, length)
        
        
        if self.transform:
            #print('transform')
            mel_feature=self.transform(mel_feature).type(torch.float32)# 데이터 0~1 정규화
            mel_feature=torch.stack([mel_feature,mel_feature,mel_feature])# 3채널로 복사.
            mel_feature = mel_feature.squeeze(dim=1)
            
            mel_feature_a=self.transform(mel_feature_a).type(torch.float32)# 데이터 0~1 정규화
            mel_feature_a=torch.stack([mel_feature_a,mel_feature_a,mel_feature_a])# 3채널로 복사.
            mel_feature_a = mel_feature_a.squeeze(dim=1)            
            
            mel_feature_i=self.transform(mel_feature_i).type(torch.float32)# 데이터 0~1 정규화
            mel_feature_i=torch.stack([mel_feature_i,mel_feature_i,mel_feature_i])# 3채널로 복사.
            mel_feature_i = mel_feature_i.squeeze(dim=1)       
            
            mel_feature_u=self.transform(mel_feature_u).type(torch.float32)# 데이터 0~1 정규화
            mel_feature_u=torch.stack([mel_feature_u,mel_feature_u,mel_feature_u])# 3채널로 복사.
            mel_feature_u = mel_feature_u.squeeze(dim=1)                 
            
            
        else:
            #print("else")
            mel_feature = torch.from_numpy(mel_feature).type(torch.float32)
            mel_feature=mel_feature.unsqueeze(0)#cnn 사용위해서 추가
            #MFCCs = MFCCs.permute(2, 0, 1)
        return mel_feature,mel_feature_a,mel_feature_i,mel_feature_u,self.classes.index(self.label[idx])
    

    
# 테스트 데이터 로더.

test_loader = torch.utils.data.DataLoader(dataset = 
                                               svd_test_set(
                                                   X_test,
                                                   classes,
                                                   transform = transforms.ToTensor(),
                                               ),
                                               batch_size = BATCH_SIZE,
                                               shuffle = True,) 


In [ ]:

for (X_test,a_test,i_test,u_test,Y_test) in test_loader:
    print("X_test : ",X_test.size(),'\na : ',a_test.size(),'\ni : ',i_test.size(),'\nu : ',u_test.size())
    print("Y_test : ",Y_test.size(),'type:',Y_test.type())
    break
    


print(Y_test[0])
plt.figure(figsize=(15,5))
plt.subplot(141)
librosa.display.specshow(X_test[0][0].numpy(), sr=sr, hop_length=hop_length)
plt.xlabel("Time")
plt.ylabel("Frequency")
plt.colorbar(format='%+2.0f dB')

plt.subplot(142)
librosa.display.specshow(a_test[0][0].numpy(), sr=sr, hop_length=hop_length)
plt.xlabel("Time")
plt.ylabel("Frequency")
plt.colorbar(format='%+2.0f dB')

plt.subplot(143)
librosa.display.specshow(i_test[0][0].numpy(), sr=sr, hop_length=hop_length)
plt.xlabel("Time")
plt.ylabel("Frequency")
plt.colorbar(format='%+2.0f dB')

plt.subplot(144)
librosa.display.specshow(u_test[0][0].numpy(), sr=sr, hop_length=hop_length)
plt.xlabel("Time")
plt.ylabel("Frequency")
plt.colorbar(format='%+2.0f dB')

#batch: 32 / 3채널 / feature수: 128/ frame수: 300   

In [ ]:
## 네트워크

class resnet18_feature(nn.Module):
    def __init__(self, embedding_dimension=128, pretrained=True):
        super().__init__()
        self.model = models.resnet18(pretrained=pretrained)
        
        # embedding
        input_features_fc_layer = self.model.fc.in_features # fc layer 채널 수 얻기
        self.model.fc = nn.Linear(input_features_fc_layer, embedding_dimension, bias=True) # fc layer 수정
        
    def forward(self, images):
        embedding = self.model(images) # embedding 생성
        return embedding

    
class resnet18_fusion(nn.Module):
    """Model modified.
    The architecture of our model is the same as standard DenseNet121
    except the classifier layer which has an additional sigmoid function.
    """

    def __init__(self, n_class=2):
        super(resnet18_fusion, self).__init__()


        model1 = resnet18_feature()
        self.phrase_model = model1
        in_features = self.phrase_model.model.fc.out_features
        

        model2 = resnet18_feature()
        self.a_model = model2

        model3 = resnet18_feature()
        self.i_model = model3

        model4 = resnet18_feature()
        self.u_model = model4
        

        self.combine = nn.Sequential(
            nn.Linear(in_features * 4, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(256,128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(128,64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(64,50),
            nn.BatchNorm1d(50),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(50,n_class)
        )



    def forward(self, phrase,a,i,u):
        phrase_feature = self.phrase_model(phrase)
        a_feature = self.a_model(a)
        i_feature = self.i_model(i)
        u_feature = self.u_model(u)      
        
        #0204 여기부터 수정.

        combine = torch.cat((phrase_feature,
                             a_feature,
                             i_feature,
                             u_feature), 1)
        combine = self.combine(combine)

        return combine
    
    
# 모델 
# pretrained


def model_initialize():
    model = resnet18_fusion(n_class=2).cuda()

    return model

model= model_initialize()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)
print(model)

In [ ]:
#confusion matrix 계산
#test set 계산.
def test_evaluate(model,test_loader):
    model.eval()
    test_loss = 0
    predictions = []
    answers = []
    #no_grad : 그래디언트 값 계산 막기.
    with torch.no_grad():
        for phrase,a,i,u,label in test_loader:
            phrase = phrase.to(DEVICE)
            a = a.to(DEVICE)
            i = i.to(DEVICE)        
            u = u.to(DEVICE)
            label = label.to(DEVICE)
            output = model(phrase,a,i,u)
            test_loss += criterion(output, label).item()
            prediction = output.max(1,keepdim=True)[1] # 가장 확률이 높은 class 1개를 가져온다.그리고 인덱스만
            answers +=label
            predictions +=prediction
            
        return predictions,answers,test_loss
            

In [ ]:
# Confusion matrix (resnet18)
# kfold의 confusion matrix는 계산 방법이 다르다.
# 모델을 각각 불러와서 test set을 평가한다.

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

cf = np.zeros((2,2))
cf_list = []
average_accuracy = 0
average_fscore = 0

for data_ind in range(1,6):

    check_path = '../../checkpoint/checkpoint_melspectro_resnet18_true_ros_'+str(data_ind)+'_300_fusion.pt'
    model.load_state_dict(torch.load(check_path))

    predictions,answers,test_loss = test_evaluate(model, test_loader)
    predictions=[ dat.cpu().numpy() for dat in predictions]
    answers=[ dat.cpu().numpy() for dat in answers]

    
    cf = confusion_matrix(answers, predictions)
    cf_list.append(cf)
    
    acc = (cf[0,0]+cf[1,1])/(cf[0,0]+cf[0,1]+cf[1,0]+cf[1,1])
    average_accuracy+=acc
    precision=cf[0,0]/(cf[0,0]+cf[1,0])
    recall=cf[0,0]/(cf[0,0]+cf[0,1])
    #fscore=2*precision*recall/(precision+recall)
    
    #fscroe macro추가
    fscore = f1_score(answers,predictions,average='macro')
    average_fscore+=fscore
    
    print('{}번 모델'.format(data_ind))
    print("Accuracy : {:.4f}% ".format(acc*100))
    print("Precision (pathology 예측한 것중 맞는 것) : {:.4f}".format(precision))
    print("recall (실제 pathology 중  예측이 맞는 것) : {:.4f}".format(recall))
    print("f score : {:.4f} ".format(fscore))
    print(cf)
    print("-----")

print("평균 acc : {:.4f}".format(average_accuracy/5))
print("평균 f1score : {:.4f}".format(average_fscore/5))



3번 모델에서  76개를 맞추어, 5개를 더 맞추게 되었다.

결과적으로 82.24 % -> 83.06%
